In [ ]:
filename = "dataframe.csv"
w = 3
wavenumber = True
num_contour = 16

In [ ]:
import numpy
import pandas
from matplotlib import pyplot, ticker

In [ ]:
data = pandas.read_csv(filename, header=0, index_col=0).T
data.index = pandas.read_csv(filename, header=None, index_col=0).iloc[0].values
xmax, xmin = 1200, 1000
data = data.iloc[:, (xmin <= data.columns) & (data.columns <= xmax)]
pyplot.rcParams.update(pyplot.rcParamsDefault)
if wavenumber: data.T.plot(legend=None, xlim=[data.columns.max(), data.columns.min()])
else: data.T.plot(legend=None, xlim=[data.columns.min(), data.columns.max()])
pyplot.show()

In [ ]:
pyplot.rcParams["figure.figsize"] = (4, 4)
pyplot.rcParams["font.size"] = 24
pyplot.rcParams["font.family"] = "serif"
pyplot.rcParams["font.serif"] = "Times New Roman"
pyplot.rcParams["xtick.bottom"] = True
pyplot.rcParams["xtick.top"] = True
pyplot.rcParams["ytick.left"] = True
pyplot.rcParams["ytick.right"] = True
pyplot.rcParams["xtick.direction"] = "in"
pyplot.rcParams["ytick.direction"] = "in"
pyplot.rcParams["xtick.major.size"] = 10
pyplot.rcParams["ytick.major.size"] = 10
pyplot.rcParams["xtick.minor.size"] = 5
pyplot.rcParams["ytick.minor.size"] = 5
pyplot.rcParams["lines.linewidth"] = 2
pyplot.rcParams["axes.linewidth"] = 2
pyplot.rcParams["xtick.major.width"] = 2
pyplot.rcParams["ytick.major.width"] = 2
pyplot.rcParams["xtick.minor.width"] = 2
pyplot.rcParams["ytick.minor.width"] = 2
pyplot.rcParams["axes.labelpad"] = 15
pyplot.rcParams["xtick.major.pad"] = 18
pyplot.rcParams["ytick.major.pad"] = 18
pyplot.rcParams["savefig.format"] = "png"
pyplot.rcParams["savefig.dpi"] = 150
pyplot.rcParams["savefig.transparent"] = False
pyplot.rcParams["savefig.facecolor"] = "white"
pyplot.rcParams["savefig.bbox"] = "tight"

In [ ]:
def contour(cont):
    x = cont.columns
    y = cont.index
    z = cont.values
    zmax = numpy.absolute(z).max()
    fig = pyplot.figure()
    ax1 = fig.add_axes((0, 0, 1, 1))
    pyplot.rcParams["lines.linewidth"] = 2
    ax1.contour(x, y, z, num_contour, cmap="bwr", vmin=-zmax, vmax=zmax)
    ax1.set_xlim(xmax, xmin)
    ax1.set_xticks(numpy.arange(xtmin, xtmax, xtdiv))
    ax1.xaxis.set_minor_locator(ticker.MultipleLocator(xtminor))
    ax1.set_xlabel("wavenumber / cm${^{-1}}$")
    ax1.set_ylim(ymin, ymax)
    ax1.set_yticks(numpy.arange(ytmin, ytmax, ytdiv))
    ax1.yaxis.set_minor_locator(ticker.MultipleLocator(ytminor))
    ax1.set_ylabel("Temperature / \N{DEGREE SIGN}C")
    ax2 = fig.add_axes((0, 1, 1, 0.5))
    ax2.set_xlim(xmax, xmin)
    ax2.set_xticks(numpy.arange(xtmin, xtmax, xtdiv))
    ax2.xaxis.set_minor_locator(ticker.MultipleLocator(xtminor))
    ax2.axes.xaxis.set_ticks([])
    ax2.axes.yaxis.set_ticks([])
    pyplot.rcParams["lines.linewidth"] = 0.5
    for i in range(len(data)): ax2.plot(data.columns, data.iloc[i], color="gray")
    pyplot.rcParams["lines.linewidth"] = 1.5
    ax2.plot(data.columns, data.iloc[0], color="blue")
    ax2.plot(data.columns, data.iloc[-1], color="red")
    pyplot.show()

In [ ]:
# synchronous PCMW2D correlation

xmin, xmax = 1000, 1200
xtmin, xtmax, xtdiv, xtminor = 1000, 1201, 50, 10

ymin, ymax = 140, 280
ytmin, ytmax, ytdiv, ytminor = 140, 281, 20, 5

sync = numpy.zeros(data.shape)
for i in range(w // 2, len(data) - w // 2):
    y = data.iloc[i - w // 2 : i + w // 2 + 1]
    y = y - y.mean()
    p = y.index - numpy.mean(y.index)
    sync[i] = p @ y / (w - 1)
sync = pandas.DataFrame(sync, index=data.index, columns=data.columns)
contour(sync)
#sync.to_csv(filename[:-4] + "_sync.csv")

In [ ]:
# asynchronous PCMW2D correlation

xmin, xmax = 1000, 1200
xtmin, xtmax, xtdiv, xtminor = 1000, 1201, 50, 10

ymin, ymax = 140, 280
ytmin, ytmax, ytdiv, ytminor = 140, 281, 20, 5

noda = numpy.zeros((w, w))
for i, j in numpy.ndindex(noda.shape):
    if i != j: noda[i, j] = 1 / numpy.pi / (j - i)
asyn = numpy.zeros(data.shape)
for i in range(w // 2, len(data) - w // 2):
    y = data.iloc[i - w // 2 : i + w // 2 + 1]
    y -= y.mean()
    p = y.index - numpy.mean(y.index)
    asyn[i] = p @ noda @ y / (w - 1)
asyn = pandas.DataFrame(asyn, index=data.index, columns=data.columns)
contour(asyn)
#asyn.to_csv(filename[:-4] + "_asyn.csv")